### Tareas específicas de la entrega 3

Hemos puesto en esta primera celda los requisitos, que son los imports y el proceso de normalización de la sección 1, para seguir con las tareas de esta entrega. 

In [ ]:
# Imports
import re
import pickle
import unicodedata
from collections import Counter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# NLP y embeddings
import spacy
from gensim.models import Word2Vec
import gensim.downloader as api
from transformers import AutoTokenizer, AutoModel

# Machine Learning
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Deep Learning
import torch
import tensorflow as tf
import tensorflow_hub as hub

df = pd.read_csv("../dataset/MTS-Dialog-TrainingSet.csv")

contraction_map = {
    "i'm": "i am",
    "you're": "you are",
    "he's": "he is",
    "she's": "she is",
    "it's": "it is",
    "that's": "that is",
    "there's": "there is",
    "don't": "do not",
    "doesn't": "does not",
    "didn't": "did not",
    "can't": "can not",
    "won't": "will not",
    "couldn't": "could not",
    "wouldn't": "would not",
    "i've": "i have",
    "we're": "we are",
    "they're": "they are",
    "i'll": "i will",
    "i'd": "i would",
    "let's": "let us",
    "what's": "what is",
    "haven't": "have not",
    "ma'am": "madam",
    "how's": "how is",
    "you've": "you have",
    "we'll": "we will",
    "hasn't": "has not",
    "you've": "you have",
    "you'll": "you will",
    "we'll": "we will",
    "hasn't": "has not",
    "how're": "how are",
    "you'd": "you would",
    "we've": "we have",
    "isn't": "is not",
    "wasn't": "was not",
    "it'll": "it will",
    "here's": "here is"
}


def expand_contractions(text):
    text = text.lower()
    for c, repl in contraction_map.items():
        text = re.sub(r"\b" + re.escape(c) + r"\b", repl, text)
    return text

def normalize_text(s, lowercase=True):
    if pd.isna(s):
        return ""

    # Normalizar unicode
    s = unicodedata.normalize("NFKC", str(s))

    # Marcadores de quién habla -> usar tokens temporales
    s = re.sub(r'\bDoctor[:\-]\s*', ' __doc__ ', s, flags=re.I)
    s = re.sub(r'\bDoctor_2[:\-]\s*', ' __doc2__ ', s, flags=re.I)
    s = re.sub(r'\bPatient[:\-]\s*', ' __pat__ ', s, flags=re.I)
    s = re.sub(r'\bGuest_family[:\-]\s*', ' __fam__ ', s, flags=re.I)
    s = re.sub(r'\bGuest_family_1[:\-]\s*', ' __fam__ ', s, flags=re.I) 
    #Si hay dos visitantes, el primero cambia de guest_family a guest_family_1, vamos a igualarlos, esté solo o no siempre será <FAMILY>
    s = re.sub(r'\bGuest_family_2[:\-]\s*', ' __fam2__ ', s, flags=re.I) 
    s = re.sub(r'\bGuest_clinician[:\-]\s*', ' __clin__ ', s, flags=re.I)
    
    # Expand contractions (suponiendo que tienes esta función)
    s = expand_contractions(s)

    # Separar puntuación
    s = re.sub(r'([.,!?;:()"\[\]])', r' \1 ', s)

    # Reducir espacios
    s = re.sub(r'\s+', ' ', s).strip()

    # Lowercase todo excepto los tags
    if lowercase:
        s = s.lower()

    # Restaurar los tags en mayúsculas
    s = s.replace('__doc__', '<DOC>')
    s = s.replace('__doc2__', '<DOC2>')
    s = s.replace('__pat__', '<PAT>')
    s = s.replace('__fam__', '<FAMILY>')
    s = s.replace('__fam2__', '<FAMILY2>')
    s = s.replace('__clin__', '<CLIN>')

    return s
 

# Versión para Embeddings NO Contextuales, y ELMo (lowercase)
df['dialog_clean'] = df['dialogue'].apply(lambda x: normalize_text(x, lowercase=True))

# Versión para BIO/ClinicalBERT (manteniendo mayúsculas)
df['dialog_clean_clinicBERT'] = df['dialogue'].apply(lambda x: normalize_text(x, lowercase=False))

# Los resúmenes
df['section_text_clean'] = df['section_text'].apply(lambda x: normalize_text(x, lowercase=True))

c:\Users\dekum\miniconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\dekum\miniconda3\envs\nlp\lib\site-packages\tensorflow_hub\__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


    Técnica de Bag of Words en Machine Learning
Usaremos CountVectorizer de scikit-learn para transformar el corpus de textos en una matriz de frecuencias de palabras, creando una representación dispersa donde cada palabra es una columna y cada documento una fila

In [2]:
# Bag-of-words (unigrams + bigrams)
cv = CountVectorizer(max_features=5000, ngram_range=(1,2))
X_bow = cv.fit_transform(df['dialog_clean'])

# TF-IDF
tfv = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X_tfidf = tfv.fit_transform(df['dialog_clean'])

    Modelo de Bag of Words
Este método cuantifica la presencia y frecuencia de las palabras en los textos, generando vectores dispersos de alta dimensión. Al transformar los datos de texto en características numéricas, se facilitará su uso en algoritmos de aprendizaje automático, permitiendo realizar tareas como clasificación, análisis de sentimientos y recomendaciones, aunque sin capturar relaciones semánticas complejas.